In [15]:
%load_ext autoreload
%autoreload 2

import pickle
import scipy.sparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import ComplementNB, GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import balanced_accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
aita_raw_df = pd.read_csv('aita_minimal_preprocess.csv', usecols=[ "post", "is_asshole"])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    aita_raw_df.post.values, 
    aita_raw_df.is_asshole.values,
    stratify = aita_raw_df.is_asshole.values,
    test_size = 0.2, 
    random_state = 448
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train,
    stratify = y_train,
    test_size = 0.2, 
    random_state = 448
)
print(X_train[0])
print(y_train[0])
print(len(X_train))

AITA for wanting to dump a girl after finding out she has an STD?So I (32m) started dating a new girl (26f) after a disgusting horrible divorce from my wife who I’ve been with for like 15 years.. 

So I was super patent and deliberate in choosing a girl to date, turning down lots of others over the last year or so of separation... Then I met this girl and she is like EVERYTHING I want in a woman, and I could really see myself having a long-term relationship with her. 

We took things slow and waited about a month to have sex, just fooling around and doing other stuff. Then when the time came she stopped me right before and told me she had been living with herpes since the age of 19.. She was very mature and responsible and understanding and said she’d give me time to think about it, but...TBH I’m fucking freaking out. I’m terrified that this could affect me and my life forever if I end up being with her.

I basically told her all of this, and broke up with her. I said that I couldn’t r

In [35]:
for min_df in [0.001, 0.005, 0.01]:
    for max_df in [0.6, 0.7, 0.8]: 
        for ngram_ceil in [3, 4]:
            vectorizer = CountVectorizer(
                lowercase=True, 
                min_df=min_df, 
                max_df=max_df, 
                ngram_range=(1,ngram_ceil)
            )
            X_train_counts = vectorizer.fit_transform(X_train)
            X_val_counts = vectorizer.transform(X_val)
            clf_comp = ComplementNB(alpha=1e-5)
            clf_comp.fit(X_train_counts, y_train)
            y_pred_comp = clf_comp.predict(X_val_counts)
            val_score = balanced_accuracy_score(y_val, y_pred_comp)
            print(f"Min df of {min_df}, Max df of {max_df}, Max Ngram {ngram_ceil} had UAR {val_score}")

Min df of 0.001, Max df of 0.6, Max Ngram 3 had UAR 0.5976658285014219
Min df of 0.001, Max df of 0.6, Max Ngram 4 had UAR 0.5971741537831234
Min df of 0.001, Max df of 0.7, Max Ngram 3 had UAR 0.5985746280929174
Min df of 0.001, Max df of 0.7, Max Ngram 4 had UAR 0.595008572455302
Min df of 0.001, Max df of 0.8, Max Ngram 3 had UAR 0.59780952107489
Min df of 0.001, Max df of 0.8, Max Ngram 4 had UAR 0.5943613343670249
Min df of 0.005, Max df of 0.6, Max Ngram 3 had UAR 0.5898002449486032
Min df of 0.005, Max df of 0.6, Max Ngram 4 had UAR 0.5887714943766607


KeyboardInterrupt: 

In [37]:
best_min_df, best_max_df, best_ceil = 0.001, 0.7, 3
vectorizer = CountVectorizer(
    lowercase=True, 
    min_df=best_min_df, 
    max_df=best_max_df, 
    ngram_range=(1,best_ceil)
)
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)
clf_comp = ComplementNB(alpha=1e-5)
clf_comp.fit(X_train_counts, y_train)
y_pred_comp = clf_comp.predict(X_test_counts)
test_score = balanced_accuracy_score(y_test, y_pred_comp)
print(f"Test set UAR: {test_score}")

Test set UAR: 0.5905575193095205
